### Want to see how many pairs of rules have at least N sessions where both rules appear at least 2 times

In [2]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils
from constants.glm_constants import *
from constants.behavioral_constants import *

import seaborn as sns

from scipy.stats import ortho_group
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import seaborn as sns
import scipy.stats


In [3]:
num_bins = 2

def get_sess_beh(row, beh_path):
    session = row.session_name
    behavior_path = beh_path.format(sess_name=session)
    beh = pd.read_csv(behavior_path)

    # filter trials 
    valid_beh = behavioral_utils.get_valid_trials(beh)
    feature_selections = behavioral_utils.get_selection_features(valid_beh)
    valid_beh = pd.merge(valid_beh, feature_selections, on="TrialNumber", how="inner")
    beh["session"] = session
    return beh

In [4]:
# Monkey S
SESSIONS_PATH = "/data/patrick_res/sessions/valid_sessions_rpe.pickle"
sessions = pd.read_pickle(SESSIONS_PATH)
beh_path = SESS_BEHAVIOR_PATH
res = pd.concat(sessions.apply(lambda x: get_sess_beh(x, beh_path), axis=1).values).reset_index()

In [5]:
num_blocks = res.groupby(["session", "CurrentRule"]).apply(lambda x: len(x.BlockNumber.unique())).reset_index()

In [6]:
num_blocks

,session,CurrentRule,0
0,20180705,CIRCLE,1
1,20180705,CYAN,3
2,20180705,ESCHER,1
3,20180705,GREEN,1
4,20180705,MAGENTA,3
...,...,...,...
305,20181010,RIPPLE,5
306,20181010,SQUARE,1
307,20181010,STAR,3
308,20181010,SWIRL,2


In [7]:
res = []
for i in range(12):
    for j in range(i + 1, 12):
        feat1 = FEATURES[i]
        feat2 = FEATURES[j]
        sess_1 = num_blocks[(num_blocks.CurrentRule == feat1) & (num_blocks[0] >= 3)].session
        sess_2 = num_blocks[(num_blocks.CurrentRule == feat2) & (num_blocks[0] >= 3)].session
        joints = sess_1[sess_1.isin(sess_2)].values
        res.append({"pair": [feat1, feat2], "sessions": joints, "num_sessions": len(joints)})
pairs = pd.DataFrame(res)

In [8]:
good_pairs = pairs[pairs.num_sessions >= 10]

In [41]:
good_pairs.to_pickle("/data/patrick_res/sessions/pairs_at_least_3blocks_10sess.pickle")

In [9]:
len(good_pairs)

10